In [1]:
# ! pip uninstall /Path/to/the/whl/file/torchlogic-0.0.1-py3-none-any.whl -y

In [2]:
# ! pip install /Path/to/the/whl/file/torchlogic-0.0.1-py3-none-any.whl

In [3]:
# ! pip install optuna

In [2]:
import warnings
warnings.filterwarnings("ignore", message="Choices for a categorical distribution should be a tuple")
warnings.filterwarnings("ignore", message="To copy construct from a tensor, it is recommended")
warnings.filterwarnings("ignore", message="IProgress not found")
warnings.filterwarnings("ignore", message="Precision is ill-defined")

import copy

import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim.swa_utils import AveragedModel

import optuna
import numpy as np
import pandas as pd
from scipy.special import softmax

from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, roc_auc_score

from torchlogic.models import BanditNRNClassifier
from torchlogic.models.mixins import ReasoningNetworkClassifierMixin

from torchlogic.modules import BanditNRNModule
from torchlogic.utils.trainers import BanditNRNTrainer

from torchlogic.nn import LukasiewiczChannelOrBlock, LukasiewiczChannelAndBlock, Predicates, ConcatenateBlocksLogic

In [5]:
# If we want the logs from Bandit-RRN training

# from carrot.logger import Logger

# log_config = 'configs/logging.yaml'
# log_dir = 'logs'
# logger = Logger.get(log_config, log_dir)

# Load Data

In [6]:
data = load_iris()
data.target[[10, 25, 50]]
data.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [7]:
data.feature_names = ["the sepal length in cm was", "the sepal width in cm was", 
                      "the petal length in cm was", "the petal width in cm was"]

# Prepare Bandit-NRN Data

A dataset for the Bandit-RRN algorithm in torchlogic must return a dictionary of the following form:

```python
{
    'features': [N_FEATURES], 'target': [N_TARGETS], 'sample_idx': [1]
}
```

- The `features` key contains a tensor of the features used for prediction.  Feature must be numeric and scaled between 0 and 1.

- The `target` key must contain a tensor of the targets, with the values of 0 or 1 for each target.

- The `sample_idx` key must contain a tensor of the row number in the data corresponding to that sample.

## Preprocess Data

In [8]:
mms = MinMaxScaler()
X = mms.fit_transform(data.data)

In [9]:
y = pd.get_dummies(data.target).astype(int).values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

###############################################################################################################################
# NOTE: The iris dataset is very small.  The validation set is particularly small in this case so we have enough training data.
# For real-world applications a larger test size is likely required.
###############################################################################################################################

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.11, random_state=42)

## Define PyTorch Dataset

In [11]:
class BanditNRNDataset(Dataset):
    def __init__(
            self,
            X: np.array,
            y: np.array
    ):
        """
        Dataset suitable for BanditRRN model from torchlogic

        Args:
            X (np.array): features data scaled to [0, 1]
            y (np.array): target data of classes 0, 1
        """
        super(BanditNRNDataset, self).__init__()
        self.X = X
        self.y = y
        self.sample_idx = np.arange(X.shape[0])  # index of samples

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        features = torch.from_numpy(self.X[idx, :]).float()
        target = torch.from_numpy(self.y[idx, :])
        return {'features': features, 'target': target, 'sample_idx': idx}

## Instantiate Datasets and Data Loaders

In [12]:
train_dataset = BanditNRNDataset(X=X_train, y=y_train)
val_dataset = BanditNRNDataset(X=X_val, y=y_val)
test_dataset = BanditNRNDataset(X=X_test, y=y_test)

In [13]:
g = torch.Generator()
g.manual_seed(0)

def create_holdout_samplers(train_dataset, pct=0.2):
    train_size = len(train_dataset)
    indices = list(range(train_size))
    np.random.seed(0)
    np.random.shuffle(indices)
    
    train_holdout_split_index = int(np.floor(pct * train_size))
    train_idx, train_holdout_idx = indices[train_holdout_split_index:], indices[:train_holdout_split_index]
    
    train_sampler = SubsetRandomSampler(train_idx)
    train_holdout_sampler = SubsetRandomSampler(train_holdout_idx)
    
    return train_sampler, train_holdout_sampler

train_sampler, train_holdout_sampler = create_holdout_samplers(train_dataset)

train_dl = DataLoader(
    train_dataset, batch_size=32, generator=g, sampler=train_sampler,
    pin_memory=False, persistent_workers=False, num_workers=0  # very important to optimize these settings in production
)
train_holdout_dl = DataLoader(
    train_dataset, batch_size=32, generator=g, sampler=train_holdout_sampler,
    pin_memory=False, persistent_workers=False, num_workers=0  # very important to optimize these settings in production
)
val_dl = DataLoader(
    val_dataset, batch_size=32, generator=g, 
    pin_memory=False, persistent_workers=False, num_workers=0  # very important to optimize these settings in production
)
test_dl = DataLoader(
    test_dataset, batch_size=32, generator=g, 
    pin_memory=False, persistent_workers=False, num_workers=0  # very important to optimize these settings in production
)

# Build Bandit-NRN model with Domain Knowledge

In [14]:
class FlowerDomainBanditNRNModule(ReasoningNetworkClassifierMixin):

    def __init__(
        self,
        feature_names,
        input_size,
        output_size,
        layer_sizes,
        n_selected_features_input,
        n_selected_features_internal,
        n_selected_features_output,
        perform_prune_quantile,
        ucb_scale,
        normal_form,
        add_negations,
        weight_init
    ):
        super(FlowerDomainBanditNRNModule, self).__init__(output_size)

        # logic induction path of model
        self.brrn = BanditNRNModule(
            input_size=input_size,
            output_size=output_size,
            layer_sizes=layer_sizes,
            feature_names=feature_names,
            n_selected_features_input=n_selected_features_input,
            n_selected_features_internal=n_selected_features_internal,
            n_selected_features_output=n_selected_features_output,
            perform_prune_quantile=perform_prune_quantile,
            ucb_scale=ucb_scale,
            normal_form=normal_form,
            add_negations=add_negations,
            weight_init=weight_init
        )

        # the model BanditNRNTrainer must have access to these layers from the extended class
        self.model = self.brrn.model
        self.output_layer = self.brrn.output_layer

        # domain knowledge path of model
        self.domain_rn = LukasiewiczChannelOrBlock(
            channels=output_size,
            in_features=input_size,
            out_features=1,
            n_selected_features=2,
            parent_weights_dimension='out_features',
            operands=Predicates(feature_names=feature_names),
            outputs_key='domain_rn'
        )

        # feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
        # target names: ['setosa', 'versicolor', 'virginica']
        # add the following knowledge:
        #     class is setosa if: petal length (cm) not >= 0.16406, OR sepal width (cm) >= 0.88541
        self.domain_rn.add_knowledge(
            channel=0,  # corresponds to the setosa class
            out_feature=0,  # only 1 output feature so the value must be 0
            input_indices=[2, 1],  # corresponds to the petal length (cm) and sepal width (cm) features
            required_thresholds=[0.16406, 0.88541],  # set the required value for the features to return a TRUE value from the logic
            required_negations=[-1.0, 1.0],  # -1.0 indicates negated logic, while 1.0 indicates non-negated logic
            freeze_knowledge=True  # freeze the logic such that it will receive no gradients
        )

        self.concatenate_logic = ConcatenateBlocksLogic(
            modules=[self.brrn.output_layer, self.domain_rn],
            outputs_key='concatenate_logic'
        )

        # NOTE: the attribute "output_layer" is a special attribute for the BanditRRN model.  There can be no other
        # modules named "output_layer" when extending BanditRRN to add domain knowledge.
        self.output_layer2 = LukasiewiczChannelAndBlock(
            channels=output_size,
            in_features=2,
            out_features=1,
            n_selected_features=2,
            parent_weights_dimension='out_features',
            operands=self.concatenate_logic,
            outputs_key='output_layer2'
        )

        self.output_layer2.add_knowledge(
            channel=0,  # corresponds to the setosa class
            out_feature=0,  # only 1 output feature so the value must be 0
            input_indices=[0, 1],  # corresponds to the concatenated logics: 0 == brrn, 1 == domain knowledge
            required_thresholds=[0.5, 0.5],  # set the required values to influence the relative importance of each logic and the required value to return TRUE
            required_negations=[1.0, 1.0],  # neither logic is negated
            freeze_knowledge=True  # freeze this layer such that the relative importance and truth values receive no gradients
        )

        self.output_layer2.add_knowledge(
            channel=1, # corresponds to the versicolor class
            out_feature=0,  # only 1 output feature so the value must be 0
            input_indices=[0, 1],  # corresponds to the concatenated logics: 0 == brrn, 1 == domain knowledge
            required_thresholds=[0.5, -1.0],  # set the required truth value.  -1.0 indicates that the domain knowledge should receive 0 weight, excluding it
            required_negations=[1.0, 1.0],  # neither logic is negated
            freeze_knowledge=True  # freeze this layer such that the relative importance and truth values receive no gradients
        )

        self.output_layer2.add_knowledge(
            channel=2,  # corresponds to the virginica class
            out_feature=0,  # only 1 output feature so the value must be 0
            input_indices=[0, 1],  # corresponds to the concatenated logics: 0 == brrn, 1 == domain knowledge
            required_thresholds=[0.5, -1.0],  # set the required truth value.  -1.0 indicates that the domain knowledge should receive 0 weight, excluding it
            required_negations=[1.0, 1.0],  # neither logic is negated
            freeze_knowledge=True # freeze this layer such that the relative importance and truth values receive no gradients
        )

    def forward(self, x):
        learned_x = self.brrn(x).unsqueeze(-1).unsqueeze(-1)
        domain_x = self.domain_rn(x)
        x = self.concatenate_logic(learned_x, domain_x)
        return self.output_layer2(x).squeeze(-1).squeeze(-1)


class FlowerDomainBanditNRNModel(BanditNRNClassifier):

    def __init__(
        self,
        target_names,
        feature_names,
        input_size,
        output_size,
        layer_sizes,
        n_selected_features_input,
        n_selected_features_internal,
        n_selected_features_output,
        perform_prune_quantile,
        ucb_scale,
        normal_form,
        delta,
        prune_strategy,
        bootstrap,
        swa,
        add_negations,
        weight_init
    ):
        super(FlowerDomainBanditNRNModel, self).__init__(
            target_names=target_names,
            feature_names=feature_names,
            input_size=input_size,
            output_size=output_size,
            layer_sizes=layer_sizes,
            n_selected_features_input=n_selected_features_input,
            n_selected_features_internal=n_selected_features_internal,
            n_selected_features_output=n_selected_features_output,
            perform_prune_quantile=perform_prune_quantile,
            ucb_scale=ucb_scale,
            normal_form=normal_form,
            delta=delta,
            prune_strategy=prune_strategy,
            bootstrap=bootstrap,
            swa=swa,
            add_negations=add_negations,
            weight_init=weight_init
        )

        self.rn = FlowerDomainBanditNRNModule(
            input_size=input_size,
            output_size=output_size,
            layer_sizes=layer_sizes,
            feature_names=feature_names,
            n_selected_features_input=n_selected_features_input,
            n_selected_features_internal=n_selected_features_internal,
            n_selected_features_output=n_selected_features_output,
            perform_prune_quantile=perform_prune_quantile,
            ucb_scale=ucb_scale,
            normal_form=normal_form,
            add_negations=add_negations,
            weight_init=weight_init
        )
        # expose required modules in current class
        self.set_modules(model=self.rn.brrn, root_layer=self.rn.output_layer2)
        
        if torch.cuda.device_count() > 1:
            self.logger.info(f"Using {torch.cuda.device_count()} GPUs!")
            self.rn = nn.DataParallel(self.rn)
        if self.USE_CUDA:
            self.logger.info(f"Using GPU")
            self.rn = self.rn.cuda()
        elif self.USE_MPS:
            self.logger.info(f"Using MPS")
            self.rn = self.rn.to('mps')

        self.USE_DATA_PARALLEL = isinstance(self.rn, torch.nn.DataParallel)

# Train Bandit-NRN Model

## Tune Hyper-parameters

In [15]:
torch.random.manual_seed(0)
np.random.seed(0)

class TuneParameters:
    
    def __init__(self, n_trials=10):
        self.best_model = None
        self.best_rn_val_performance = 0.0
        self.n_trials = n_trials

    def _objective(self, trial):

        ########################################################################################################################
        # NOTE: These hyper-parameter settings are specific to the iris flower dataset.  For information on generally useful
        # ranges of hyper-parameters and their descriptions see our documentation: 
        ########################################################################################################################

        # Set Parameters
        
        ## Reinforced Reasoning Network Parameters
        layer_sizes = trial.suggest_categorical('layer_sizes', [(2, ), (3, ), (5, ), (10, ), 
                                                                (2, 2), (3, 3), (5, 5), (10, 10), 
                                                                (2, 2, 2), (3, 3, 3), (5, 5, 5), (10, 10, 10)])
        n_selected_features_input = trial.suggest_int('n_selected_features_input', low=2, high=3)
        n_selected_features_internal = trial.suggest_int('n_selected_features_internal', low=2, high=min(3, min(layer_sizes)))
        n_selected_features_output = trial.suggest_int('n_selected_features_output', low=2, high=min(3, layer_sizes[-1]))
        perform_prune_plateau_count = trial.suggest_int('perform_prune_plateau_count', low=1, high=1)
        perform_prune_quantile = trial.suggest_float('perform_prune_quantile', low=0.1, high=0.9)
        increase_prune_plateau_count = trial.suggest_int('increase_prune_plateau_count', low=0, high=20)
        increase_prune_plateau_count_plateau_count = trial.suggest_int('increase_prune_plateau_count_plateau_count', low=10, high=30)
        ucb_scale = trial.suggest_float('ucb_scale', low=1.0, high=2.0)
        normal_form = trial.suggest_categorical('normal_form', ['dnf', 'cnf'])
        prune_strategy = trial.suggest_categorical('prune_strategy', ['class', 'logic'])
        delta = trial.suggest_float('delta', low=2.0, high=2.0)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])
        swa = trial.suggest_categorical('swa', [True, False])
        add_negations = trial.suggest_categorical('add_negations', [True, False])
        weight_init = trial.suggest_float('weight_init', low=0.01, high=1.0)

        ## Optimizer Parameters

        ### Learning Rate
        learning_rate = trial.suggest_float('learning_rate', low=0.01, high=0.2)

        ### L1 Regularization
        use_l1 = trial.suggest_categorical('use_l1', [True, False])
        if use_l1:
            l1_lambda = trial.suggest_float('l1_lambda', low=0.00001, high=0.1)
        else:
            l1_lambda = 0

        ### Weight Decay Regularization
        use_weight_decay = trial.suggest_categorical('use_weight_decay', [True, False])
        if use_weight_decay:
            weight_decay = trial.suggest_float('weight_decay', low=0.00001, high=0.1)
        else:
            weight_decay = 0

        ### Lookahead Optimization
        use_lookahead = trial.suggest_categorical('use_lookahead', [True, False])
        if use_lookahead:
            lookahead_steps = trial.suggest_int('lookahead_steps', low=5, high=10, step=1)
            lookahead_steps_size = trial.suggest_float('lookahead_steps_size', low=0.5, high=0.8)
        else:
            lookahead_steps = 0
            lookahead_steps_size = 0

        ### Data Augmentation
        # augment = trial.suggest_categorical('augment', ['CM', 'MU', 'AT', None])
        augment = trial.suggest_categorical('augment', ['CM', 'MU', None])  # excluding Adversarial Learning because it fails on Jupyter Notebooks
        if augment is not None:
            augment_alpha = trial.suggest_float('augment_alpha', low=0.0, high=1.0)
        else:
            augment_alpha = 0

        ### Early Stopping
        early_stopping_plateau_count = trial.suggest_int('early_stopping_plateau_count', low=5, high=10, step=1)
        
        ## Scheulder parameters
        t_0 = trial.suggest_int('T_0', low=2, high=10, step=1)
        t_mult = trial.suggest_int('T_mult', low=1, high=3, step=1)

        # init model
        model = FlowerDomainBanditNRNModel(
            target_names=[x + '_label' for x in data.target_names],
            feature_names=data.feature_names,
            input_size=len(data.feature_names),
            output_size=len(data.target_names),
            layer_sizes=layer_sizes,
            n_selected_features_input=n_selected_features_input,
            n_selected_features_internal=n_selected_features_internal,
            n_selected_features_output=n_selected_features_output,
            perform_prune_quantile=perform_prune_quantile,
            ucb_scale=ucb_scale,
            normal_form=normal_form,
            delta=delta,
            prune_strategy=prune_strategy,
            bootstrap=bootstrap,
            swa=swa,
            add_negations=add_negations,
            weight_init=weight_init
        )

        epochs = 100
        accumulation_steps = 1
        optimizer = optim.AdamW(model.rn.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=t_0, T_mult=t_mult)
        trainer = BanditNRNTrainer(
            model=model,
            loss_func=nn.BCELoss(),
            optimizer=optimizer,
            scheduler=scheduler,
            epochs=epochs,
            accumulation_steps=accumulation_steps,
            l1_lambda=l1_lambda,
            early_stopping_plateau_count=early_stopping_plateau_count,
            perform_prune_plateau_count=perform_prune_plateau_count,
            increase_prune_plateau_count=increase_prune_plateau_count,
            increase_prune_plateau_count_plateau_count=increase_prune_plateau_count_plateau_count,
            lookahead_steps=lookahead_steps,
            lookahead_steps_size=lookahead_steps_size,
            augment=augment,
            augment_alpha=augment_alpha,
            class_independent=True
        )

        # train model
        trainer.train(train_dl, train_holdout_dl, evaluation_metric=roc_auc_score, multi_class=True)
        trainer.set_best_state()

        # evaluate model
        predictions, targets = trainer.model.predict(val_dl)
        rn_val_performance = trainer.model.evaluate(
            predictions=predictions,
            labels=targets
        )

        if rn_val_performance > self.best_rn_val_performance:
            self.best_rn_val_performance = rn_val_performance
            self.best_model = copy.copy(trainer.model)
            self.best_model.rn = copy.deepcopy(trainer.model.rn)

        return rn_val_performance
    
    def tune(self):
        # 3. Create a study object and optimize the objective function.
        sampler = optuna.samplers.TPESampler(multivariate=True, group=True, seed=42)
        study = optuna.create_study(direction='maximize', sampler=sampler)
        study.optimize(self._objective, n_trials=self.n_trials)
        return self.best_model

In [ ]:
best_model = TuneParameters(25).tune()

In [17]:
predictions, targets = best_model.predict(val_dl)
rn_val_performance = best_model.evaluate(
    predictions=predictions,
    labels=targets
)
class_predictions = predictions.eq(predictions.max(axis=1), axis=0).astype(int)
predictions_probs = pd.DataFrame(softmax(predictions, axis=1), columns=data.target_names)  # CrossEntropyLoss takes logits, therefore predictions are logits
print("Validation AUC:\n\n", rn_val_performance)

Validation AUC:

 1.0


In [18]:
predictions, targets = best_model.predict(test_dl)
rn_test_performance = best_model.evaluate(
    predictions=predictions,
    labels=targets
)
class_predictions = predictions.eq(predictions.max(axis=1), axis=0).astype(int)
predictions_probs = pd.DataFrame(softmax(predictions, axis=1), columns=data.target_names)  # CrossEntropyLoss takes logits, therefore predictions are logits
print("Test AUC:\n\n", rn_test_performance)

Test AUC:

 0.9845679012345679


In [19]:
predictions_probs

,setosa,versicolor,virginica
0,0.216590,0.338428,0.444982
1,0.372779,0.250341,0.376880
2,0.193638,0.343143,0.463219
3,0.216086,0.339175,0.444739
4,0.220344,0.336452,0.443203
5,0.337681,0.273963,0.388355
6,0.231823,0.334332,0.433846
7,0.208448,0.341145,0.450407
8,0.195688,0.347636,0.456676
9,0.222099,0.337779,0.440122


In [20]:
class_predictions

,probs_setosa,probs_versicolor,probs_virginica
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
5,0,0,1
6,0,0,1
7,0,0,1
8,0,0,1
9,0,0,1


# Inspecting the Model

### Global Explain

A global explanation prints the logic learned for each class.  The `quantile` parameter is the percent of the model you would like to be explained.

We represent our features as values scaled between 0 and 1.  Therefore, we intepret the explanations to mean that large values for a particular feature represent `truthiness` of a predicate, while small values represent `falseness` of a predicate.

For example, the following logic for the class `setosa`:

```
A flower is in the setosa class because: 
AND(
    NOT(AND(
            sepal width (cm) >= 0.77405,
            petal length (cm) >= 0.4397)),
    NOT(OR(
            AND(
            sepal width (cm) >= 0.67788,
            petal length (cm) >= 0.20122),
            NOT(sepal width (cm) >= 0.48579))))
```

The `logic` from above is intepreted as:

```
When BOTH of the following are true the class is "setosa":
    1. The flower has a sepal width below the transformed value of 0.77, and has a petal length below the transformed value of 0.44.
    2. The flower has a sepal width below the transformed value of 0.68 and a petal length below the transformed value of 0.20; OR the flower has a sepal width above the transformed value of 0.49.
```

In [21]:
print(best_model.explain(
    quantile=1.0,
    required_output_thresholds=np.array(1.0),
    explain_type='both',
    print_type='logical-natural', 
    explanation_prefix="A flower is in the",
    target_names=data.target_names,
    ignore_uninformative=False,
    rounding_precision=5,
    # inverse_transform=mms.inverse_transform
))

A flower is in the setosa because: 


All the following are true: 
	Any of the following are true: 
		All the following are true: 
			It was not true that 
				The petal width in cm was >= 0.18528
			It was not true that 
				The sepal width in cm was >= 0.56848
		All the following are true: 
			The sepal length in cm was >= 0.79044
			The sepal width in cm was >= 0.69027
	Any of the following are true: 
		The sepal width in cm was >= 0.88541
		It was not true that 
			The petal length in cm was >= 0.16406

A flower is in the versicolor because: 


It was not true that 
	All the following are true: 
		The sepal width in cm was >= 0.75596
		It was not true that 
			The sepal length in cm was >= 0.28744

A flower is in the virginica because: 


Any of the following are true: 
	It was not true that 
		It was not true that 
			The sepal length in cm was >= 1.0
	All the following are true: 
		The petal length in cm was >= 1.0
		The petal width in cm was >= 1.0


### Did the model use our Domain Knowledge?

We can see from the results above that the setosa class is using our logic directly.

```
A flower is in the setosa class because:

    ...

    ANY of the following are TRUE:
        - NOT petal length (cm) greater than 0.164,
        - sepal width (cm) greater than 0.885,
```

# Build Bandit-NRN with Uncertain Domain Knowledge

In [22]:
class FlowerDomainBanditNRNModule(ReasoningNetworkClassifierMixin):

    def __init__(
        self,
        feature_names,
        input_size,
        output_size,
        layer_sizes,
        n_selected_features_input,
        n_selected_features_internal,
        n_selected_features_output,
        perform_prune_quantile,
        ucb_scale,
        normal_form,
        add_negations,
        weight_init
    ):
        super(FlowerDomainBanditNRNModule, self).__init__(output_size)

        # logic induction path of model
        self.brrn = BanditNRNModule(
            input_size=input_size,
            output_size=output_size,
            layer_sizes=layer_sizes,
            feature_names=feature_names,
            n_selected_features_input=n_selected_features_input,
            n_selected_features_internal=n_selected_features_internal,
            n_selected_features_output=n_selected_features_output,
            perform_prune_quantile=perform_prune_quantile,
            ucb_scale=ucb_scale,
            normal_form=normal_form,
            add_negations=add_negations,
            weight_init=weight_init
        )

        # the model BanditRRNTrainer must have access to these layers from the extended class
        self.model = self.brrn.model
        self.output_layer = self.brrn.output_layer

        # domain knowledge path of model
        self.domain_rn = LukasiewiczChannelOrBlock(
            channels=output_size,
            in_features=input_size,
            out_features=1,
            n_selected_features=2,
            parent_weights_dimension='out_features',
            operands=Predicates(feature_names=feature_names),
            outputs_key='domain_rn'
        )

        # feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
        # target names: ['setosa', 'versicolor', 'virginica']
        # add the following knowledge:
        #     class is setosa if: petal length (cm) not >= 0.16406, OR sepal width (cm) >= 0.88541
        self.domain_rn.add_knowledge(
            channel=0,  # corresponds to the setosa class
            out_feature=0,  # only 1 output feature so the value must be 0
            input_indices=[2, 1],  # corresponds to the petal length (cm) and sepal width (cm) features
            required_thresholds=[0.05, 0.05],  # set the required value for the features to return a TRUE value from the logic
            required_negations=[-1.0, 1.0],  # -1.0 indicates negated logic, while 1.0 indicates non-negated logic
            freeze_knowledge=False  # DONT freeze the logic such that it will receive gradients
        )

        self.concatenate_logic = ConcatenateBlocksLogic(
            modules=[self.brrn.output_layer, self.domain_rn],
            outputs_key='concatenate_logic'
        )

        # NOTE: the attribute "output_layer" is a special attribute for the BanditRRN model.  There can be no other
        # modules named "output_layer" when extending BanditRRN to add domain knowledge.
        self.output_layer2 = LukasiewiczChannelAndBlock(
            channels=output_size,
            in_features=2,
            out_features=1,
            n_selected_features=2,
            parent_weights_dimension='out_features',
            operands=self.concatenate_logic,
            outputs_key='output_layer2'
        )

        self.output_layer2.add_knowledge(
            channel=0,  # corresponds to the setosa class
            out_feature=0,  # only 1 output feature so the value must be 0
            input_indices=[0, 1],  # corresponds to the concatenated logics: 0 == brrn, 1 == domain knowledge
            required_thresholds=[0.5, 0.5],  # set the required values to influence the relative importance of each logic and the required value to return TRUE
            required_negations=[1.0, 1.0],  # neither logic is negated
            freeze_knowledge=False  # DONT freeze this layer such that the relative importance and truth values receive gradients
        )

        self.output_layer2.add_knowledge(
            channel=1, # corresponds to the versicolor class
            out_feature=0,  # only 1 output feature so the value must be 0
            input_indices=[0, 1],  # corresponds to the concatenated logics: 0 == brrn, 1 == domain knowledge
            required_thresholds=[0.5, -1.0],  # set the required truth value.  -1.0 indicates that the domain knowledge should receive 0 weight, excluding it
            required_negations=[1.0, 1.0],  # neither logic is negated
            freeze_knowledge=False  # DONT freeze this layer such that the relative importance and truth values receive gradients
        )

        self.output_layer2.add_knowledge(
            channel=2,  # corresponds to the virginica class
            out_feature=0,  # only 1 output feature so the value must be 0
            input_indices=[0, 1],  # corresponds to the concatenated logics: 0 == brrn, 1 == domain knowledge
            required_thresholds=[0.5, -1.0],  # set the required truth value.  -1.0 indicates that the domain knowledge should receive 0 weight, excluding it
            required_negations=[1.0, 1.0],  # neither logic is negated
            freeze_knowledge=False # freeze this layer such that the relative importance and truth values receive gradients
        )

    def forward(self, x):
        learned_x = self.brrn(x).unsqueeze(-1).unsqueeze(-1)
        domain_x = self.domain_rn(x)
        x = self.concatenate_logic(learned_x, domain_x)
        return self.output_layer2(x).squeeze(-1).squeeze(-1)


class FlowerDomainBanditNRNModel(BanditNRNClassifier):

    def __init__(
        self,
        target_names,
        feature_names,
        input_size,
        output_size,
        layer_sizes,
        n_selected_features_input,
        n_selected_features_internal,
        n_selected_features_output,
        perform_prune_quantile,
        ucb_scale,
        normal_form,
        delta,
        prune_strategy,
        bootstrap,
        swa,
        add_negations,
        weight_init
    ):
        super(FlowerDomainBanditNRNModel, self).__init__(
            target_names=target_names,
            feature_names=feature_names,
            input_size=input_size,
            output_size=output_size,
            layer_sizes=layer_sizes,
            n_selected_features_input=n_selected_features_input,
            n_selected_features_internal=n_selected_features_internal,
            n_selected_features_output=n_selected_features_output,
            perform_prune_quantile=perform_prune_quantile,
            ucb_scale=ucb_scale,
            normal_form=normal_form,
            delta=delta,
            prune_strategy=prune_strategy,
            bootstrap=bootstrap,
            swa=swa,
            add_negations=add_negations,
            weight_init=weight_init
        )

        self.rn = FlowerDomainBanditNRNModule(
            input_size=input_size,
            output_size=output_size,
            layer_sizes=layer_sizes,
            feature_names=feature_names,
            n_selected_features_input=n_selected_features_input,
            n_selected_features_internal=n_selected_features_internal,
            n_selected_features_output=n_selected_features_output,
            perform_prune_quantile=perform_prune_quantile,
            ucb_scale=ucb_scale,
            normal_form=normal_form,
            add_negations=add_negations,
            weight_init=weight_init
        )   
            
        # expose required modules in current class
        self.set_modules(model=self.rn.brrn, root_layer=self.rn.output_layer2)

        if torch.cuda.device_count() > 1:
            self.logger.info(f"Using {torch.cuda.device_count()} GPUs!")
            self.rn = nn.DataParallel(self.rn)
        if self.USE_CUDA:
            self.logger.info(f"Using GPU")
            self.rn = self.rn.cuda()
        elif self.USE_MPS:
            self.logger.info(f"Using MPS")
            self.rn = self.rn.to('mps')

        self.USE_DATA_PARALLEL = isinstance(self.rn, torch.nn.DataParallel)

# Train Bandit-NRN Model to Modify Domain Knowledge

In [23]:
torch.random.manual_seed(0)
np.random.seed(0)

class TuneParameters:
    
    def __init__(self, n_trials=10):
        self.best_model = None
        self.best_rn_val_performance = 0.0
        self.n_trials = n_trials

    def _objective(self, trial):

        ########################################################################################################################
        # NOTE: These hyper-parameter settings are specific to the iris flower dataset.  For information on generally useful
        # ranges of hyper-parameters and their descriptions see our documentation: 
        ########################################################################################################################

        # Set Parameters
        
        ## Reinforced Reasoning Network Parameters
        layer_sizes = trial.suggest_categorical('layer_sizes', [(2, ), (3, ), (5, ), (10, )])
        n_selected_features_input = trial.suggest_int('n_selected_features_input', low=2, high=3)
        n_selected_features_internal = trial.suggest_int('n_selected_features_internal', low=2, high=min(3, min(layer_sizes)))
        n_selected_features_output = trial.suggest_int('n_selected_features_output', low=2, high=min(3, layer_sizes[-1]))
        perform_prune_plateau_count = trial.suggest_int('perform_prune_plateau_count', low=1, high=1)
        perform_prune_quantile = trial.suggest_float('perform_prune_quantile', low=0.1, high=0.9)
        increase_prune_plateau_count = trial.suggest_int('increase_prune_plateau_count', low=0, high=20)
        increase_prune_plateau_count_plateau_count = trial.suggest_int('increase_prune_plateau_count_plateau_count', low=10, high=30)
        ucb_scale = trial.suggest_float('ucb_scale', low=1.0, high=2.0)
        normal_form = trial.suggest_categorical('normal_form', ['dnf', 'cnf'])
        prune_strategy = trial.suggest_categorical('prune_strategy', ['class', 'logic'])
        delta = trial.suggest_float('delta', low=2.0, high=2.0)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])
        swa = trial.suggest_categorical('swa', [False])
        add_negations = trial.suggest_categorical('add_negations', [True, False])
        weight_init = trial.suggest_float('weight_init', low=0.01, high=1.0)

        ## Optimizer Parameters

        ### Learning Rate
        learning_rate = trial.suggest_float('learning_rate', low=0.01, high=0.2)

        ### L1 Regularization
        use_l1 = trial.suggest_categorical('use_l1', [True, False])
        if use_l1:
            l1_lambda = trial.suggest_float('l1_lambda', low=0.00001, high=0.1)
        else:
            l1_lambda = 0

        ### Weight Decay Regularization
        use_weight_decay = trial.suggest_categorical('use_weight_decay', [True, False])
        if use_weight_decay:
            weight_decay = trial.suggest_float('weight_decay', low=0.00001, high=0.1)
        else:
            weight_decay = 0

        ### Lookahead Optimization
        use_lookahead = trial.suggest_categorical('use_lookahead', [True, False])
        if use_lookahead:
            lookahead_steps = trial.suggest_int('lookahead_steps', low=5, high=10, step=1)
            lookahead_steps_size = trial.suggest_float('lookahead_steps_size', low=0.5, high=0.8)
        else:
            lookahead_steps = 0
            lookahead_steps_size = 0

        ### Data Augmentation
        # augment = trial.suggest_categorical('augment', ['CM', 'MU', 'AT', None])
        augment = trial.suggest_categorical('augment', ['CM', 'MU', None])  # excluding Adversarial Learning because it fails on Jupyter Notebooks
        if augment is not None:
            augment_alpha = trial.suggest_float('augment_alpha', low=0.0, high=1.0)
        else:
            augment_alpha = 0

        ### Early Stopping
        early_stopping_plateau_count = trial.suggest_int('early_stopping_plateau_count', low=5, high=10, step=1)
        
        ## Scheulder parameters
        t_0 = trial.suggest_int('T_0', low=2, high=10, step=1)
        t_mult = trial.suggest_int('T_mult', low=1, high=3, step=1)

        # init model
        model = FlowerDomainBanditNRNModel(
            target_names=[x + '_label' for x in data.target_names],
            feature_names=data.feature_names,
            input_size=len(data.feature_names),
            output_size=len(data.target_names),
            layer_sizes=layer_sizes,
            n_selected_features_input=n_selected_features_input,
            n_selected_features_internal=n_selected_features_internal,
            n_selected_features_output=n_selected_features_output,
            perform_prune_quantile=perform_prune_quantile,
            ucb_scale=ucb_scale,
            normal_form=normal_form,
            delta=delta,
            prune_strategy=prune_strategy,
            bootstrap=bootstrap,
            swa=swa,
            add_negations=add_negations,
            weight_init=weight_init
        )

        epochs = 100
        accumulation_steps = 1
        optimizer = optim.AdamW(model.rn.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=t_0, T_mult=t_mult)
        trainer = BanditNRNTrainer(
            model=model,
            loss_func=nn.BCELoss(),
            optimizer=optimizer,
            scheduler=scheduler,
            epochs=epochs,
            accumulation_steps=accumulation_steps,
            l1_lambda=l1_lambda,
            early_stopping_plateau_count=early_stopping_plateau_count,
            perform_prune_plateau_count=perform_prune_plateau_count,
            increase_prune_plateau_count=increase_prune_plateau_count,
            increase_prune_plateau_count_plateau_count=increase_prune_plateau_count_plateau_count,
            lookahead_steps=lookahead_steps,
            lookahead_steps_size=lookahead_steps_size,
            augment=augment,
            augment_alpha=augment_alpha,
            class_independent=True
        )

        # train model
        trainer.train(train_dl, train_holdout_dl, evaluation_metric=roc_auc_score, multi_class=True)
        trainer.set_best_state()

        # evaluate model
        predictions, targets = trainer.model.predict(val_dl)
        rn_val_performance = trainer.model.evaluate(
            predictions=predictions,
            labels=targets
        )

        if rn_val_performance > self.best_rn_val_performance:
            self.best_rn_val_performance = rn_val_performance
            self.best_model = copy.copy(trainer.model)
            self.best_model.rn = copy.deepcopy(trainer.model.rn)

        return rn_val_performance
    
    def tune(self):
        # 3. Create a study object and optimize the objective function.
        sampler = optuna.samplers.TPESampler(multivariate=True, group=True, seed=0)
        study = optuna.create_study(direction='maximize', sampler=sampler)
        study.optimize(self._objective, n_trials=self.n_trials)
        return self.best_model

In [ ]:
tuner = TuneParameters(25)
best_model = tuner.tune()

In [25]:
predictions, targets = best_model.predict(val_dl)
rn_val_performance = best_model.evaluate(
    predictions=predictions,
    labels=targets
)
class_predictions = predictions.eq(predictions.max(axis=1), axis=0).astype(int)
predictions_probs = pd.DataFrame(softmax(predictions, axis=1), columns=data.target_names)
print("Validation AUC:\n\n", rn_val_performance)

Validation AUC:

 0.9940476190476191


In [26]:
predictions, targets = best_model.predict(test_dl)
rn_test_performance = best_model.evaluate(
    predictions=predictions,
    labels=targets
)
class_predictions = predictions.eq(predictions.max(axis=1), axis=0).astype(int)
predictions_probs = pd.DataFrame(softmax(predictions, axis=1), columns=data.target_names)
print("Test AUC:\n\n", rn_test_performance)

Test AUC:

 0.9259259259259259


In [27]:
print(best_model.explain(
    quantile=1.0,
    required_output_thresholds=np.array(1.0),
    explain_type='both',
    print_type='logical-natural', 
    explanation_prefix="A flower is in the",
    target_names=data.target_names,
    ignore_uninformative=False,
    rounding_precision=5,
    # inverse_transform=mms.inverse_transform
))

A flower is in the setosa because: 


All the following are true: 
	Any of the following are true: 
		The sepal width in cm was >= 0.05
		It was not true that 
			The petal length in cm was >= 0.21005
	Any of the following are true: 
		The sepal length in cm was >= 1.0
		It was not true that 
			The sepal width in cm was >= 0.0

A flower is in the versicolor because: 


All the following are true: 
	Any of the following are true: 
		The petal length in cm was >= 1.0
		The sepal length in cm was >= 1.0
		It was not true that 
			The petal length in cm was >= 0.0
		It was not true that 
			The sepal length in cm was >= 0.0
	Any of the following are true: 
		The petal width in cm was >= 1.0
		It was not true that 
			The petal width in cm was >= 0.0

A flower is in the virginica because: 


All the following are true: 
	Any of the following are true: 
		The petal length in cm was >= 1.0
		The petal width in cm was >= 1.0
		It was not true that 
			The petal length in cm was >= 0.0
		It wa

### Was our domain knowledge modified?

From the explanation above we can see that the model modified our logic.

Our original logic that encoded our domain knowledge about the setosa flower was the following:

```
ANY of the following are TRUE:
    - NOT petal length (cm) greater than 0.05, 
    - sepal width (cm) greater than 0.05
```

After training, the model learned to modify our logic such that it better supported the induced logic from the supervised learning stream in our Reasoning Network.
The new modified logic is as follows.

```
ANY of the following are TRUE: 
    - NOT petal length (cm) greater than 0.21008, 
    - sepal width (cm) greater than 0.04999
```

# Sample explanation

In [28]:
print(best_model.explain_samples(
    val_dataset[0]['features'].unsqueeze(0),
    quantile=1.0,
    target_names=data.target_names, 
    explain_type='both',
    sample_explanation_prefix="The flower was in the",
    print_type='natural',
    ignore_uninformative=True,
    rounding_precision=5,
    inverse_transform=mms.inverse_transform
))

0: The flower was in the virginica because: 

There are several scenarios that must be met for this prediction to hold true.  The first scenario is as follows.  It was NOT true the petal length in cm was greater than or equal to 5.71806, or the petal length in cm was greater than or equal to 5.1176, or the petal width in cm was greater than or equal to 2.16408.

The next scenario that must be met is as follows.  It was NOT true the sepal length in cm was greater than or equal to 7.77171, or the petal length in cm was greater than or equal to 4.3714, or the sepal length in cm was greater than or equal to 5.76421.

The next scenario that must be met is as follows.  The petal length in cm was greater than or equal to 5.059, and the petal length in cm was greater than or equal to 5.13937, and it was NOT true the petal length in cm was greater than and it was NOT true equal to 5.78646


In [29]:
print(best_model.explain_samples(
    val_dataset[0]['features'].unsqueeze(0),
    quantile=1.0,
    target_names=data.target_names, 
    explain_type='both',
    sample_explanation_prefix="The flower was in the",
    print_type='logical-natural',
    ignore_uninformative=True,
    rounding_precision=5,
    inverse_transform=mms.inverse_transform
))

0: The flower was in the virginica because: 


All the following are true: 
	It was not true that 
		Any of the following are true: 
			The petal length in cm was >= 5.78646
			It was not true that 
				The petal length in cm was >= 5.059
			It was not true that 
				The petal length in cm was >= 5.13937
	Any of the following are true: 
		The petal length in cm was >= 5.1176
		The petal width in cm was >= 2.16408
		It was not true that 
			The petal length in cm was >= 5.71806
	Any of the following are true: 
		The petal length in cm was >= 4.3714
		The sepal length in cm was >= 5.76421
		It was not true that 
			The sepal length in cm was >= 7.77171


In [30]:
print(best_model.explain_samples(
    val_dataset[0]['features'].unsqueeze(0),
    quantile=1.0,
    target_names=data.target_names, 
    explain_type='both',
    sample_explanation_prefix="The flower was in the",
    print_type='logical',
    ignore_uninformative=True,
    rounding_precision=5,
    inverse_transform=mms.inverse_transform
))

0: The flower was in the virginica because: 


AND 
	NOT 
		OR 
			The petal length in cm was >= 5.78646
			NOT 
				The petal length in cm was >= 5.059
			NOT 
				The petal length in cm was >= 5.13937
	OR 
		The petal length in cm was >= 5.1176
		The petal width in cm was >= 2.16408
		NOT 
			The petal length in cm was >= 5.71806
	OR 
		The petal length in cm was >= 4.3714
		The sepal length in cm was >= 5.76421
		NOT 
			The sepal length in cm was >= 7.77171
